In [1]:
import argparse
import gym
import numpy as np
import torch
from sac import SAC
import robel
from updated_env import DKittyOrientRandomDynamics

In [2]:
env = DKittyOrientRandomDynamics([2.]*8) #gym.make("DKittyOrientRandomDynamics-v0")
env.max_episode_steps = 80
env.seed(42)
env.action_space.seed(42)
env._max_episode_steps = 80 ####### TO CHECK ITs consistent with PEARL (Arsen)
torch.manual_seed(42)
np.random.seed(42)

In [3]:
class Args:
    def __init__(self):
        self.policy = "Gaussian"
        self.eval = True
        self.gamma = 0.99
        self.tau = 0.005
        self.lr = 0.0003
        self.alpha = 0.2
        self.automatic_entropy_tuning = False
        self.batch_size = 256
        self.num_steps = 1000001
        self.hidden_size = 256
        self.update_per_step = 1
        self.start_steps = 10000
        self.target_update_interval = 1
        self.replay_size = 1000000
        self.cuda = True
args = Args()

In [4]:
agent = SAC(env.observation_space.shape[0], env.action_space, args)

In [5]:
agent.load_model("./models/sac_actor_DKittyOrientUpdateRandomDynamics_",

                 "./models/sac_critic_DKittyOrientUpdateRandomDynamics_")

Loading models from ./models/sac_actor_DKittyOrientUpdateRandomDynamics_ and ./models/sac_critic_DKittyOrientUpdateRandomDynamics_


In [6]:
avg_reward = 0.
avg_success = 0.
episodes = 100
max_test_r = -np.inf
for _  in range(episodes):
    state = env.reset()
    env.render()
    episode_reward = 0
    episode_success = 0
    done = False
    for i in range(80):
        action = agent.select_action(state, evaluate=True)
        next_state, reward, done, info = env.step(action)
        episode_reward += reward
        state = next_state
        if info['score/success']:
            episode_success = 1
    avg_success += episode_success       
    avg_reward += episode_reward
avg_reward /= episodes
avg_success /= episodes

print("----------------------------------------")
print("Test Episodes: {}, Avg. Success: {}".format(episodes, round(avg_success, 2)))
print("----------------------------------------")
print("Test Episodes: {}, Avg. Reward: {}".format(episodes, round(avg_reward, 2)))
print("----------------------------------------")
env.close()

Creating window glfw


KeyboardInterrupt: 